#**INTELIGENCIA ARTIFICIAL AVANZADA 2PC**
#**DATA FALLECIDOS SINADEF**


In [33]:
import pandas as pd

In [34]:
fal_sin=pd.read_csv("/content/fallecidos_sinadef.csv", sep=';' , encoding='latin-1',header=2).iloc[:,1:-4]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
fal_sin.head()

,TIPO SEGURO,SEXO,EDAD,TIEMPO EDAD,ESTADO CIVIL,NIVEL DE INSTRUCCIÓN,COD# UBIGEO DOMICILIO,PAIS DOMICILIO,DEPARTAMENTO DOMICILIO,PROVINCIA DOMICILIO,DISTRITO DOMICILIO,FECHA,AÑO,MES,TIPO LUGAR,INSTITUCION,MUERTE VIOLENTA,NECROPSIA,DEBIDO A (CAUSA A),CAUSA A (CIE-X),DEBIDO A (CAUSA B),CAUSA B (CIE-X),DEBIDO A (CAUSA C),CAUSA C (CIE-X),DEBIDO A (CAUSA D),CAUSA D (CIE-X),DEBIDO A (CAUSA E),CAUSA E (CIE-X),DEBIDO A (CAUSA F),CAUSA F (CIE-X)
0,SIS,MASCULINO,4,AÑOS,SOLTERO,IGNORADO,92-33-21-04-01-000,PERU,SAN MARTIN,MARISCAL CACERES,JUANJUI,2020-01-17,2020,1,EESS,GOBIERNO REGIONAL,SIN REGISTRO,NO SE REALIZÓ NECROPSIA,INSUFICIENCIA RESPIRATORIA AGUDA,J960,NEUMONIA SEVERA,J159,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,PARALISIS CEREBRAL INFANTIL,G809,SIN REGISTRO,SIN REGISTRO
1,SIS,FEMENINO,86,AÑOS,SOLTERO,PRIMARIA INCOMPLETA,92-33-21-03-13-000,PERU,SAN MARTIN,LAMAS,TABALOSOS,2020-01-03,2020,1,DOMICILIO,SIN REGISTRO,SIN REGISTRO,NO SE REALIZÓ NECROPSIA,INFARTO AGUDO DEL MIOCARDIO SIN OTRA ESPECIFIC...,I219,HIPERLIPIDEMIA NO ESPECIFICADA,E785,ATEROSCLEROSIS GENERALIZADA Y LA NO ESPECIFICADA,I709,HIPERTENSION ESENCIAL,I10X,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
2,IGNORADO,MASCULINO,38,AÑOS,IGNORADO,IGNORADO,93-32-00-00-00-000,FILIPINAS,EXTRANJERO,EXTRANJERO,EXTRANJERO,2020-01-19,2020,1,OTRO,SIN REGISTRO,SIN REGISTRO,SI SE REALIZÓ NECROPSIA,EDEMA Y CONGESTION CEREBRAL Y PULMONAR,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
3,USUARIO,FEMENINO,64,AÑOS,SOLTERO,PRIMARIA COMPLETA,92-33-14-01-37-000,PERU,LIMA,LIMA,SAN JUAN DE LURIGANCHO,2020-01-06,2020,1,DOMICILIO,SIN REGISTRO,SIN REGISTRO,SI SE REALIZÓ NECROPSIA,TUBERCULOSIS PULMONAR,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
4,IGNORADO,MASCULINO,20,AÑOS,SOLTERO,IGNORADO,92-33-24-01-02-000,PERU,CALLAO,CALLAO,BELLAVISTA,2020-01-13,2020,1,EESS,GOBIERNO REGIONAL,HOMICIDIO,SI SE REALIZÓ NECROPSIA,SHOCK HIPOVOLEMICO,SIN REGISTRO,TAPONAMIENTO CARDIACO,SIN REGISTRO,TRAUMA TORACICO ABIERTO POR PROYECTIL DE ARMA ...,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO


In [36]:
fal_sin["TIPO SEGURO"].value_counts(dropna=False)

SIS              253923
ESSALUD          156001
IGNORADO          94457
USUARIO            9863
OTROS              9504
PRIVADOS           8699
SANIDAD PNP        8361
SANIDAD NAVAL      1533
SANIDAD EP         1377
SANIDAD FAP        1363
EXONERADO           900
SOAT                 29
SIN REGISTRO          1
Name: TIPO SEGURO, dtype: int64

In [37]:
def  TIPO_SEGURO_FE(x):
    if(x=="SANIDAD PNP"):
      return "SANIDAD"
    if(x=="SANIDAD NAVAL"):
      return "SANIDAD"
    if(x=="SANIDAD EP"):
      return "SANIDAD"
    if(x=="SANIDAD FAP"):
      return "SANIDAD"
    if(x=="SOAT"):
      return "PRIVADOS"
    if(x=="SIS"):
      return "PUBLICO"
    if(x=="ESSALUD"):
      return "PUBLICO"
    return "OTROS"

In [38]:
fal_sin["TIPO SEGURO_FE"]=fal_sin["TIPO SEGURO"].apply(TIPO_SEGURO_FE)

In [39]:
fal_sin["TIPO SEGURO_FE"].value_counts()

PUBLICO     409924
OTROS       123424
SANIDAD      12634
PRIVADOS        29
Name: TIPO SEGURO_FE, dtype: int64

In [40]:
fal_sin["SEXO"].value_counts(dropna=False)

MASCULINO        306169
FEMENINO         239771
INDETERMINADO        67
SIN REGISTRO          4
Name: SEXO, dtype: int64

In [41]:
fal_sin["TIEMPO EDAD"].value_counts(dropna=False)

AÑOS            527537
DIAS              7541
MESES             7047
HORAS             2641
MINUTOS            805
IGNORADO           269
SIN REGISTRO       138
SEGUNDOS            33
Name: TIEMPO EDAD, dtype: int64

In [42]:
fal_sin["TIEMPO EDAD"].value_counts()

AÑOS            527537
DIAS              7541
MESES             7047
HORAS             2641
MINUTOS            805
IGNORADO           269
SIN REGISTRO       138
SEGUNDOS            33
Name: TIEMPO EDAD, dtype: int64

In [44]:
def TIEMPO_EDAD_FE_ANIO(x) :
    if (x=="AÑOS"):
      return 1
    else:
      return 0           

In [45]:
fal_sin["factor"]=fal_sin["TIEMPO EDAD"].apply(TIEMPO_EDAD_FE_ANIO)

In [46]:
fal_sin[fal_sin["EDAD"]=="SIN REGISTRO"]=str(fal_sin.loc[fal_sin["EDAD"]!="SIN REGISTRO","EDAD"].astype(int).mean())

In [47]:
fal_sin["EDAD"]=fal_sin["EDAD"].astype(float).astype(int)

In [48]:
fal_sin["EDAD"].describe()

count    546011.000000
mean         66.322334
std          23.435579
min           0.000000
25%          55.000000
50%          72.000000
75%          83.000000
max        3017.000000
Name: EDAD, dtype: float64

In [49]:
fal_sin["TIEMPO"]=fal_sin["EDAD"]*fal_sin["factor"]

In [50]:
fal_sin["TIEMPO"]

0          4
1         86
2         38
3         64
4         20
          ..
546006    82
546007    72
546008    86
546009    89
546010     0
Name: TIEMPO, Length: 546011, dtype: object

In [51]:
fal_sin.loc[(fal_sin["DEBIDO A (CAUSA A)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA B)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA C)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA D)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA E)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA F)"].str.contains("covid", case=False)) ,"Target"]=1
                

In [52]:
fal_sin["Target"]=fal_sin["Target"].fillna(0)

In [53]:
fal_sin["Target"]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
546006    1.0
546007    1.0
546008    0.0
546009    0.0
546010    0.0
Name: Target, Length: 546011, dtype: float64

In [54]:
fal_sin.columns

Index(['TIPO SEGURO', 'SEXO', 'EDAD', 'TIEMPO EDAD', 'ESTADO CIVIL',
       'NIVEL DE INSTRUCCIÓN', 'COD# UBIGEO DOMICILIO', 'PAIS DOMICILIO',
       'DEPARTAMENTO DOMICILIO', 'PROVINCIA DOMICILIO', 'DISTRITO DOMICILIO',
       'FECHA', 'AÑO', 'MES', 'TIPO LUGAR', 'INSTITUCION', 'MUERTE VIOLENTA',
       'NECROPSIA', 'DEBIDO A (CAUSA A)', 'CAUSA A (CIE-X)',
       'DEBIDO A (CAUSA B)', 'CAUSA B (CIE-X)', 'DEBIDO A (CAUSA C)',
       'CAUSA C (CIE-X)', 'DEBIDO A (CAUSA D)', 'CAUSA D (CIE-X)',
       'DEBIDO A (CAUSA E)', 'CAUSA E (CIE-X)', 'DEBIDO A (CAUSA F)',
       'CAUSA F (CIE-X)', 'TIPO SEGURO_FE', 'factor', 'TIEMPO', 'Target'],
      dtype='object')

In [55]:
fal_sin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546011 entries, 0 to 546010
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   TIPO SEGURO             546011 non-null  object 
 1   SEXO                    546011 non-null  object 
 2   EDAD                    546011 non-null  int64  
 3   TIEMPO EDAD             546011 non-null  object 
 4   ESTADO CIVIL            546011 non-null  object 
 5   NIVEL DE INSTRUCCIÓN    546011 non-null  object 
 6   COD# UBIGEO DOMICILIO   546011 non-null  object 
 7   PAIS DOMICILIO          546011 non-null  object 
 8   DEPARTAMENTO DOMICILIO  546011 non-null  object 
 9   PROVINCIA DOMICILIO     546011 non-null  object 
 10  DISTRITO DOMICILIO      546011 non-null  object 
 11  FECHA                   546011 non-null  object 
 12  AÑO                     546011 non-null  object 
 13  MES                     546011 non-null  object 
 14  TIPO LUGAR          

In [56]:
data_weka=fal_sin[['TIPO SEGURO', 
'SEXO', 
'ESTADO CIVIL',
'NIVEL DE INSTRUCCIÓN', 
'TIPO SEGURO_FE', 
'TIEMPO',
'PAIS DOMICILIO',
'DEPARTAMENTO DOMICILIO', 
'PROVINCIA DOMICILIO', 
'DISTRITO DOMICILIO',
'AÑO', 
'MES', 
'TIPO LUGAR', 
'INSTITUCION', 
'MUERTE VIOLENTA',
'NECROPSIA', 
'Target']]

In [59]:
data_weka.to_csv("fallecidos_procesado.csv")

In [64]:
data_weka.isnull().sum()

TIPO SEGURO               0
SEXO                      0
ESTADO CIVIL              0
NIVEL DE INSTRUCCIÓN      0
TIPO SEGURO_FE            0
TIEMPO                    0
PAIS DOMICILIO            0
DEPARTAMENTO DOMICILIO    0
PROVINCIA DOMICILIO       0
DISTRITO DOMICILIO        0
AÑO                       0
MES                       0
TIPO LUGAR                0
INSTITUCION               0
MUERTE VIOLENTA           0
NECROPSIA                 0
Target                    0
dtype: int64

In [63]:
data_weka["Target"].value_counts(dropna=False)

0.0    478398
1.0     67613
Name: Target, dtype: int64